In [71]:
# Import all the libraries
import pandas as pd
import glob
import os
import re
from pprint import pprint

In [72]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\NV\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]

# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'caucus'])
]


In [73]:
print("General files:")
for f in general_files:
    print(f)


General files:
C:\Huy Phan\College\VoterTurnout\data\NV\20161108__nv__general__precinct.csv


In [74]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\NV\20160220__nv__democratic__caucus__precinct.csv


In [75]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"] == "President"]
            
        # Combine precinct as county + precinct
        df["precinct"] = df["jurisdiction"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
            df = df[~df['precinct'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]   

        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]               
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)
pri_combined_df

,jurisdiction,precinct,office,party,candidate,votes
0,Carson City,CARSON CITY101,President,Democratic,Hillary Clinton,3
1,Carson City,CARSON CITY103,President,Democratic,Hillary Clinton,4
2,Carson City,CARSON CITY105,President,Democratic,Hillary Clinton,3
3,Carson City,CARSON CITY107,President,Democratic,Hillary Clinton,3
4,Carson City,CARSON CITY109,President,Democratic,Hillary Clinton,4
...,...,...,...,...,...,...
6795,White Pine,WHITE PINE5,President,Democratic,Uncommitted,0
6796,White Pine,WHITE PINE6,President,Democratic,Uncommitted,0
6797,White Pine,WHITE PINE7,President,Democratic,Uncommitted,0
6798,White Pine,WHITE PINE8,President,Democratic,Uncommitted,0


In [77]:
# Checking any suspicious precinct, county
suspicious_precincts = pri_combined_df[
    pri_combined_df["precinct"].isna() |
    pri_combined_df["precinct"].str.strip().str.upper().str.contains(r"\w*TOTAL\w*", na=False) |
    pri_combined_df["precinct"].str.strip().str.upper().isin(["NONE", "NAN"]) |
    pri_combined_df["jurisdiction"].isna() |
    pri_combined_df["jurisdiction"].str.strip().str.upper().str.contains(r"\w*TOTAL\w*", na=False) |
    pri_combined_df["jurisdiction"].str.strip().str.upper().isin(["NONE", "NAN"])
]

In [78]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]

primary_data

,precinct,party,candidate,votes
0,CARSON CITY101,Democratic,Hillary Clinton,3
1,CARSON CITY103,Democratic,Hillary Clinton,4
2,CARSON CITY105,Democratic,Hillary Clinton,3
3,CARSON CITY107,Democratic,Hillary Clinton,3
4,CARSON CITY109,Democratic,Hillary Clinton,4
...,...,...,...,...
6795,WHITE PINE5,Democratic,Uncommitted,0
6796,WHITE PINE6,Democratic,Uncommitted,0
6797,WHITE PINE7,Democratic,Uncommitted,0
6798,WHITE PINE8,Democratic,Uncommitted,0


In [79]:
# Viewing candidate data
primary_data["candidate"].value_counts(dropna=False)

candidate
Hillary Clinton    1700
Bernie Sanders     1700
Other              1700
Uncommitted        1700
Name: count, dtype: int64

In [80]:
# Cleaning Candidates

# Turning all primary data to uppercase
primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()

unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"UNCOMMITTED", r"OTHER",r"Total Votes Cast",r"No Preference"
]

pattern = "|".join(unwanted_keywords)

# Assuming candidate column is already string and uppercase
primary_data = primary_data[~primary_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

#fIXING DE LA FUENTE
primary_data.loc[
    (
        primary_data["candidate"].str.contains("rocky.*fuente", case=False, na=False) |
        primary_data["candidate"].str.contains("LA FUENTE", case=False, na=False) |
        primary_data["candidate"].str.contains("\w*Fuque\w*", case=False, na=False)
    ),
    "candidate"
] = "LA FUENTE"


primary_data.loc[:, "candidate"] = primary_data["candidate"].replace({
    "Donald I. Trump": "Donald J. Trump",
})

primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
# # Fixing the , candidate
# primary_data["candidate"] = (
#     primary_data["candidate"].str.split(",")
#     .str[0]
#     .str.strip()
#     )


# #Selecting only last name
primary_data["candidate"] = primary_data["candidate"].str.split().str[-1]

# Fixing Christie
primary_data.loc[
    primary_data["candidate"].str.contains("CHRISTLE", case=False, na=False),
    "candidate"
] = "CHRISTIE"

# Fixing KASICH
primary_data.loc[
    primary_data["candidate"].str.contains("KAISCH", case=False, na=False),
    "candidate"
] = "CHRISTIE"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("O'MALLEY", case=False, na=False),
    "candidate"
] = "O'MAILEY"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("PATAKL", case=False, na=False),
    "candidate"
] = "PATAKI"

# Fixing O'MALLEY
primary_data.loc[
    primary_data["candidate"].str.contains("RUBLO", case=False, na=False),
    "candidate"
] = "RUBIO"

# # Fixing Huckabee
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ABEE\w*", case=False, na=False),
#     "candidate"
# ] = "HUCKABEE"

# # Fixing Fiorina
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ORINA\W*", case=False, na=False),
#     "candidate"
# ] = "FIORINA"

# # Fixing Gray
# primary_data.loc[
#     primary_data["candidate"].str.contains("GREY", case=False, na=False),
#     "candidate"
# ] = "GRAY"

# # Fixing Wilson
# primary_data.loc[
#     primary_data["candidate"].str.contains("WISON", case=False, na=False),
#     "candidate"
# ] = "WILSON"


# # Fixing separator 
# primary_data["candidate"] = (
#     primary_data["candidate"]
#     .str.split(r"\s*(?:and|/|&|–|-|\+)\s*", n=1, expand=True)[0]
#     .str.strip()
#     .str.upper()
# )

# # Fixing McMullin

# primary_data.loc[
#     (
#         primary_data["candidate"].str.contains("MCMULLIN", case=False, na=False) |
#         primary_data["candidate"].str.contains("EVAN MCMULLEN", case=False, na=False)
#     ),
#     "candidate"
# ] = "MCMULLIN"

# primary_data.loc[
#     primary_data["candidate"].str.contains("De La Fuen", case=False, na=False),
#     "candidate"
# ] = "FUENTE D"

# primary_data.loc[:,"candidate"] = (
#     primary_data["candidate"].str.split().str[0].str.upper()
# )

primary_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\2567635960.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\2567635960.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\2567635960.py:43: SettingWithCopyWarning: 
A value is trying to be set on a cop

candidate
CLINTON    1700
SANDERS    1700
Name: count, dtype: int64

In [81]:
# Viewing Party
primary_data["party"].value_counts(dropna=False)

party
Democratic    3400
Name: count, dtype: int64

In [ ]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown

#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
# primary_data["party"] = primary_data.apply(
#     lambda row: fill_party_from_data(row, primary_data),
#     axis=1
# )

primary_data["party"].value_counts(dropna=False)


party
Republican    23205
Democrat       8420
Democratic      505
Name: count, dtype: int64

In [82]:
# Cleaning Party
# Turning all general data party to uppercase
primary_data["party"] = primary_data["party"].astype(str).str.upper()

# primary_data["party"] = primary_data.apply(
#     lambda row: fill_party_from_data(row, primary_data),
#     axis=1
# )
primary_data["party"] = (
    primary_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEMOCRAT":"DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GREEN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK"
    })
)
primary_data["party"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\453441538.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = primary_data["party"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\453441538.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["party"] = (


party
DEM    3400
Name: count, dtype: int64

In [83]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"]
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\2521670330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data.loc[:,"candidate_column"] = (


candidate_column,precinct,pri_dem_CLINTON,pri_dem_SANDERS
0,CARSON CITY101,3,4
1,CARSON CITY103,4,3
2,CARSON CITY105,3,2
3,CARSON CITY107,3,3
4,CARSON CITY109,4,4
...,...,...,...
1695,WHITE PINE6,7,6
1696,WHITE PINE7,4,0
1697,WHITE PINE8,2,1
1698,WHITE PINE9,1,1


In [84]:
# Process general files
general_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)
        
        # Combine precinct as county + precinct
        df["precinct"] = df["precinct"].astype(str).str.extract(r'(\d+)', expand=False)
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()
        df["county"] = df["county"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
        
        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]      
            
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"] == "President"]
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        general_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(general_df_list, ignore_index=True)

# Checking any suspicious precinct
suspicious_precincts = gen_combined_df[
    gen_combined_df["precinct"].str.strip().str.upper().isin(["TOTAL", "NAN"])
]
gen_combined_df


,county,precinct,office,district,candidate,votes,party
0,CARSON CITY,CARSON CITY101,President,NaN,"CASTLE, DARRELL",5,IAP
1,CARSON CITY,CARSON CITY101,President,NaN,"CLINTON, HILLARY",354,DEM
2,CARSON CITY,CARSON CITY101,President,NaN,"DE LA FUENTE, ROQUE ROCKY""""",3,NPP
3,CARSON CITY,CARSON CITY101,President,NaN,"JOHNSON, GARY",39,LPN
4,CARSON CITY,CARSON CITY101,President,NaN,None Of These Candidates,39,NaN
...,...,...,...,...,...,...,...
11985,WHITE PINE,WHITE PINE99,President,NaN,"CLINTON, HILLARY",0,DEM
11986,WHITE PINE,WHITE PINE99,President,NaN,"DE LA FUENTE, ROQUE ROCKY""""",0,NPP
11987,WHITE PINE,WHITE PINE99,President,NaN,"JOHNSON, GARY",0,LPN
11988,WHITE PINE,WHITE PINE99,President,NaN,None Of These Candidates,0,NaN


In [85]:
# Select only the relevant columns
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]

general_data

,precinct,party,candidate,votes
0,CARSON CITY101,IAP,"CASTLE, DARRELL",5
1,CARSON CITY101,DEM,"CLINTON, HILLARY",354
2,CARSON CITY101,NPP,"DE LA FUENTE, ROQUE ROCKY""""",3
3,CARSON CITY101,LPN,"JOHNSON, GARY",39
4,CARSON CITY101,NaN,None Of These Candidates,39
...,...,...,...,...
11985,WHITE PINE99,DEM,"CLINTON, HILLARY",0
11986,WHITE PINE99,NPP,"DE LA FUENTE, ROQUE ROCKY""""",0
11987,WHITE PINE99,LPN,"JOHNSON, GARY",0
11988,WHITE PINE99,NaN,None Of These Candidates,0


In [86]:
# Viewing candidate data
general_data["candidate"] = general_data["candidate"].astype(str).str.upper()
general_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\1743146100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


candidate
CLINTON, HILLARY               2002
TRUMP, DONALD J.               2002
JOHNSON, GARY                  2001
NONE OF THESE CANDIDATES       2000
CASTLE, DARRELL                1995
DE LA FUENTE, ROQUE ROCKY""    1990
Name: count, dtype: int64

In [87]:
# Cleaning Candidates

# Turning all general data to uppercase
general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"\w*WRITE[\s-]\w*",
    r"NAN",
    r"UNCERTIFIED",
    r"UNVERIFIED",
    r"NONE OF THE ABOVE",
    r"LBT",
    r"DEM",
    r"REP",
    r"GRN",
    r"NAN",
    r"\w*CANDIDATES\w*"
]

# Fixing FUENTE
general_data.loc[
    general_data["candidate"].str.contains("\w*FUENTE\w*", case=False, na=False),
    "candidate"
] = "FUENTE"
pattern = "|".join(unwanted_keywords)

# FILTER OUT TRASH WORDS
general_data = general_data[~general_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

# # Fixing Darrell Castle"
# general_data.loc[
#     general_data["candidate"].str.contains("\w*ASTLE\w*", case=False, na=False),
#     "candidate"
# ] = "CASTLE"

# general_data.loc[
#     general_data["candidate"].str.contains("\w*ATUREN\w*", case=False, na=False),
#     "candidate"
# ] = "MATUREN"

# # Fixing mixed president + vice_president by /
# general_data["candidate"] = (
#     general_data["candidate"].str.split("/")
#     .str[0]
#     .str.strip()
#     )

# #Selecting only last name
# general_data["candidate"] = general_data["candidate"].str.split().str[-1]

# Fixing the , candidate
general_data["candidate"] = (
    general_data["candidate"].str.split(",")
    .str[0]
    .str.strip()
    )




# # Fixing Cubbler
# general_data.loc[
#     ( 
#         general_data["candidate"].str.contains("\w*UBBIER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*UBLER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*OBBLER\w*", case=False, na=False),
#     "candidate")
# ] = "CUBBLER"


# # Fixing Kotlikoff
# general_data.loc[
#     general_data["candidate"].str.contains("\w*TIKOFF\w*", case=False, na=False),
#     "candidate"
# ] = "KOTLIKOFF"

# # Fixing Valdivia
# general_data.loc[
#     general_data["candidate"].str.contains("\w*VALDIVA\W*", case=False, na=False),
#     "candidate"
# ] = "VALDIVIA"

# # Fixing HOEFLING
# general_data.loc[
#     general_data["candidate"].str.contains("\w*HOEFFLING", case=False, na=False),
#     "candidate"
# ] = "HOEFLING"


# # Fixing McMullin
# general_data.loc[
#     general_data["candidate"].str.contains("MCMULLEN", case=False, na=False) ,
#     "candidate"
# ] = "MCMULLIN"

# Fixing LASTNAME + First name Initial
# general_data.loc[:,"candidate"] = (
#     general_data["candidate"].str.split().str[0].str.upper()
# )


# #Selecting only last name
general_data["candidate"] = general_data["candidate"].str.split().str[-1]

general_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\1012402545.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = general_data["candidate"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\1012402545.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = (
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\1012402545.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

candidate
CLINTON    2002
TRUMP      2002
JOHNSON    2001
CASTLE     1995
FUENTE     1990
Name: count, dtype: int64

In [88]:
# Viewing Party
general_data["party"].value_counts(dropna=False)

party
DEM    2002
REP    2002
LPN    2001
IAP    1995
NPP    1990
Name: count, dtype: int64

In [89]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown
    

#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)

    # Filter and combine only known party entries (exclude 'UNK')
    new_data = (
        df[["candidate", "party"]]
        .dropna()
        .query('party != "UNK"')
        .drop_duplicates()
    )

    # Merge with master and remove duplicates by candidate
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)

    # Filter and combine only known party entries (exclude 'UNK')
    new_data = (
        df[["candidate", "party"]]
        .dropna()
        .query('party != "UNK"')
        .drop_duplicates()
    )

    # Merge with master and remove duplicates by candidate
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
# general_data["party"] = general_data.apply(
#     lambda row: fill_party_from_data(row, general_data),
#     axis=1
# )

# # Fill remaining party using general master CSV
# master_party_df = pd.read_csv(r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv") # USE YOUR OWN ADDRESS
# general_data = fill_party_from_master(general_data, master_party_df)
general_data["party"].value_counts(dropna=False)


party
DEM    2002
REP    2002
LPN    2001
IAP    1995
NPP    1990
Name: count, dtype: int64

In [90]:
# Cleaning Party
# Turning all general data party to uppercase
general_data["party"] = general_data["party"].astype(str).str.upper()

general_data["party"] = (
    general_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEMOCRAT":"DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GRN",
        "GREEN AND RAINBOW":"GRN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LIBERTARIN":"LIB",
        "LPN":"LIB",
        "LBT": "LIB",
        "L": "LIB",
        "IAP":"CON",
        "CONSTITUTION": "CON",
        "AMERICAN DELTA":"AMD",
        "PROHIBITION":"PRO",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK",
        "NPA":"IND",
        "UST":"CON",
        "NPP":"IND",
        "(WRITE-IN)":"IND"
    })
)

general_data["party"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\1264349760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["party"] = general_data["party"].astype(str).str.upper()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\1264349760.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["party"] = (


party
DEM    2002
REP    2002
LIB    2001
CON    1995
IND    1990
Name: count, dtype: int64

In [91]:
# UPDATE MASTER FILE, CAREFUL
update_master_candidate_party(general_data, r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv")

In [92]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21640\1829951435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate_column"] = (


candidate_column,precinct,gen_con_CASTLE,gen_dem_CLINTON,gen_ind_FUENTE,gen_lib_JOHNSON,gen_rep_TRUMP
0,CARSON CITY101,5,354,3,39,419
1,CARSON CITY103,6,392,1,25,476
2,CARSON CITY105,8,375,1,39,608
3,CARSON CITY107,8,376,1,39,617
4,CARSON CITY109,11,482,0,47,606
...,...,...,...,...,...,...
1972,WHITE PINE7,2,25,0,6,115
1973,WHITE PINE8,3,11,2,9,192
1974,WHITE PINE88,0,0,0,0,0
1975,WHITE PINE9,0,46,0,5,48


In [93]:
# Merge
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")

# Convert DEM primary columns to numeric and calculate total
dem_cols = combined.filter(like="pri_dem_").columns
combined[dem_cols] = combined[dem_cols].apply(pd.to_numeric, errors="coerce")
combined["dem_primary_total"] = combined[dem_cols].sum(axis=1)

# Convert general election columns to numeric and calculate total
gen_cols = combined.filter(like="gen_").columns
combined[gen_cols] = combined[gen_cols].apply(pd.to_numeric, errors="coerce")
combined["general_total"] = combined[gen_cols].sum(axis=1)

# Convert all numeric columns to int
for col in combined.columns[1:]:
    combined[col] = pd.to_numeric(combined[col], errors='coerce').fillna(0).astype(int)

# Identify filtered-out precincts
primary_precincts = set(primary_result["precinct"])
general_precincts = set(general_result["precinct"])
combined_precincts = set(combined["precinct"])

primary_filtered_out = primary_precincts - combined_precincts
general_filtered_out = general_precincts - combined_precincts




In [94]:


combined.to_csv("NV.csv", index=False)
# Step 3: Identify unmatched precincts and retrieve full rows
combined_precincts = set(combined["precinct"])

# Rows in primary_result but not in combined
primary_filtered_out = primary_result[~primary_result["precinct"].isin(combined_precincts)]
primary_filtered_out.to_csv("NV_primary_filtered.csv", index=False)

# Rows in general_result but not in combined
general_filtered_out = general_result[~general_result["precinct"].isin(combined_precincts)]
general_filtered_out.to_csv("NV_general_filtered.csv", index=False)

In [95]:
print(f"primary: {len(primary_precincts)}, general: {len(general_precincts)}, combined: {len(combined)}")

primary: 1700, general: 1977, combined: 1213


In [ ]:
print(f"primary: {len(pri_combined_df['jurisdiction'].unique())}, general: {len(gen_combined_df['county'].unique())}")


primary: 18, general: 17


In [ ]:
pri_combined_df['jurisdiction'].unique()

array(['Carson City', 'Churchill', 'Clark', 'Douglas', 'Elko',
       'Esmeralda', 'Eureka', 'Humboldt', 'Lander', 'Lincoln', 'Lyon',
       'Mineral', 'Nye', 'ye', 'Pershing', 'Storey', 'Washoe',
       'White Pine'], dtype=object)

In [ ]:
gen_combined_df['county'].unique()

array(['CARSON CITY', 'CHURCHILL', 'CLARK', 'DOUGLAS', 'ELKO',
       'ESMERALDA', 'EUREKA', 'HUMBOLDT', 'LANDER', 'LINCOLN', 'LYON',
       'MINERAL', 'NYE', 'PERSHING', 'STOREY', 'WASHOE', 'WHITE PINE'],
      dtype=object)

In [ ]:
primary_counties = set(pri_combined_df['jurisdiction'].dropna().str.strip().str.upper())
general_counties = set(gen_combined_df['county'].dropna().str.strip().str.upper())

diff = primary_counties - general_counties
print(f"Counties in primary but not in general: {len(diff)}")
print(diff)


Counties in primary but not in general: 1
{'YE'}
